## Problem 1

Find the absolute minimum of the function

\begin{equation}
    f(x) = f(x_1, x_2) = x_1 e^{-{x_1}^2 -{x_2}^2}
\end{equation}

in the domain $x \in \mathbb{R}^2$ (unconstrained problem).

The initial point is $x^0 = (-0.6, -0.3)$.

In [3]:
## gradient-descent method (1st order)

import sympy as sp
import numpy as np
import pandas as pd
from sympy.utilities.lambdify import lambdify
from scipy.optimize import minimize_scalar

# Define the gradient descent optimization function
def GradientDescent(f, x0, ε, J):
    x1, x2 = sp.symbols('x1 x2')

    # Calculate the gradient and Hessian matrix
    grad_f = sp.Matrix([sp.diff(f, x1), sp.diff(f, x2)])

    # Create lambda functions for the objective function and gradient
    f_lambda = lambdify((x1, x2), f, 'numpy')
    grad_lambda = lambdify((x1, x2), grad_f, 'numpy')

    # Initialize variables
    x = np.array(x0, dtype=float)
    num_iterations = 0
    prev_grad = 0

    optimization_data = []

    λ = None  # Initialize λ

    # Main gradient descent loop
    while num_iterations <= J:
        # Calculate the gradient at the current point
        g = np.array(grad_lambda(x[0], x[1]), dtype=float)
        grad_norm = np.linalg.norm(g)
        fx = f_lambda(x[0], x[1])


        if abs(grad_norm - prev_grad) <= ε or num_iterations == J:
            break

        prev_grad = grad_norm

        # Define a function for g(λ) that takes λ as an argument and returns f(x + λs)
        # λ minimices this function (line search)
        g_lambda = lambda lam: f_lambda(x[0] - lam * g[0], x[1] - lam * g[1])

        # Use minimize_scalar to find the optimal λ
        result = minimize_scalar(g_lambda)
        λ = float(result.x)

        # Append data to the list as a dictionary, separating x into x1 and x2
        optimization_data.append({'Iteration': num_iterations, 'x1': x[0], 'x2': x[1], 'f(x)': fx, '|∇f(x)|': grad_norm, 'λ': λ})
        # Update x using the optimal λ and the gradient
        x = x - np.reshape(λ * g, x.shape)
        num_iterations += 1

    # Create a DataFrame to store optimization data and add last iteration
    optimization_data.append({'Iteration': num_iterations, 'x1': x[0], 'x2': x[1], 'f(x)': fx, '|∇f(x)|': grad_norm, 'λ': λ})
    optimization_data_df = pd.DataFrame(optimization_data)

    # Print results
    print("Iterations performed:", num_iterations)
    print("The point that minimizes the function f is x* = [" + str(x[0]) + ', ' + str(x[1]) + '], such that f(x*) = ' + str(fx))

    return fx, optimization_data_df

# Define the symbols and objective function
x1, x2 = sp.symbols('x1 x2')
def f(x1_val, x2_val):
    return x1_val * sp.exp(-x1_val**2 - x2_val**2)

# Set tolerance, initial point, and search direction
ε = 1e-15
J = 100
x0 = [-0.6, -0.3]

# Call the gradient descent function
result, optimization_data = GradientDescent(f(x1, x2), x0, ε, J)

# Print the optimization data DataFrame
print('\n')
print(optimization_data)



Iterations performed: 71
The point that minimizes the function f is x* = [-0.7071067811880666, 2.7211058675555406e-15], such that f(x*) = -0.42888194248035344


    Iteration        x1            x2      f(x)       |∇f(x)|             λ
0           0 -0.600000 -3.000000e-01 -0.382577  2.908032e-01  9.266659e-01
1           1 -0.765443 -8.728743e-02 -0.422814  1.202278e-01  7.456717e-01
2           2 -0.694677 -3.224738e-02 -0.428303  3.499490e-02  8.455856e-01
3           3 -0.712845 -8.889514e-03 -0.428820  1.241814e-02  7.200949e-01
4           4 -0.705786 -3.399510e-03 -0.428875  3.694085e-03  8.464618e-01
..        ...       ...           ...       ...           ...           ...
67         67 -0.707107  4.134883e-14 -0.428882  1.106592e-10  1.095448e+00
68         68 -0.707107  2.495982e-15 -0.428882  9.729950e-11 -1.396333e+00
69         69 -0.707107  5.485481e-15 -0.428882  3.303754e-10  5.875089e-01
70         70 -0.707107  2.721106e-15 -0.428882  2.606028e-12  1.947564e-11
71 